In [ ]:
import zmq
import numpy as np
import threading

import zmq
import numpy as np

def create_push_socket(ip, port):
    context = zmq.Context()
    socket = context.socket(zmq.PUSH)
    socket.connect(f"tcp://{ip}:{port}")
    
    return socket

def create_pull_socket(ip, port):
    context = zmq.Context()
    socket = context.socket(zmq.PULL)
    socket.bind(f"tcp://{ip}:{port}")
    
    return socket

def send_number_matrix(push_socket, number, matrix):

    msg = {"number": number, "matrix": matrix.tolist()}

    push_socket.send_json(msg)
    
def receive_number(pull_socket):
    msg = pull_socket.recv_json()
    
    return float(msg["number"])
    
    
def aggregate_numbers_matrices(pull_socket, n):
    local_matrices = []
    local_numbers = []

    for _ in range(n):
        msg = pull_socket.recv_json()
        matrix = np.array(msg["matrix"])
        number = float(msg["number"])
        local_matrices.append(matrix)
        local_numbers.append(number)

    mean_matrix = np.mean(local_matrices, axis=0)
    mean_number = np.mean(local_numbers)

    return mean_number, mean_matrix

def start_aggregation_loop(pull_socket, n, callback):
    def loop():
        while True:
            result = aggregate_numbers_matrices(pull_socket, n)
            callback(result)

    t = threading.Thread(target=loop, daemon=True)
    t.start()
    
    
def on_aggregated(result):
    # TODO: Send back!
    mean_number, mean_matrix = result
    print("Aggregation complete")
    print("Mean number:", mean_number)
    print("Mean matrix:\n", mean_matrix)

In [5]:
pull_socket = create_pull_socket('localhost', 9205)
start_aggregation_loop(pull_socket, n=2, callback=on_aggregated)

Aggregation complete
Mean number: 2.0
Mean matrix:
 [[0.30453645 0.81297287]]
Aggregation complete
Mean number: 3.0
Mean matrix:
 [[0.51056123 0.59371875]]
Aggregation complete
Mean number: 4.0
Mean matrix:
 [[0.40571598 0.69017968]]


In [2]:
pull = create_pull_socket('localhost', 9205)

In [3]:
aggregate_numbers_matrices(pull, 2)

(np.float64(2.0), array([[0.56839408, 0.33640212]]))